# Controls Example

If you want to run this notebook on your Raspberry Pi, but don't have Jupyter Notebook, you can install it with `pip`.

`pip install jupyter`

Then to run it, simply enter `jupyter-notebook` into the Raspberry Pi terminal
and then navigate the `picamera2/examples/notebooks` folder to find this notebook.

Note: If running in chromium, you may see better performance by turning off hardware acceleration in the chromium settings.

***

## Setup

In [1]:
from picamera2.picamera2 import *
import time

picam2 = Picamera2()

[0:18:02.268175580] [2137]  INFO Camera camera_manager.cpp:293 libcamera v0.0.0+3548-a11d63f9
[0:18:02.311149512] [2151]  WARN CameraSensorProperties camera_sensor_properties.cpp:163 No static properties available for 'imx477'
[0:18:02.311188660] [2151]  WARN CameraSensorProperties camera_sensor_properties.cpp:165 Please consider updating the camera sensor properties database
[0:18:02.311207530] [2151] ERROR CameraSensor camera_sensor.cpp:591 'imx477 10-001a': Camera sensor does not support test pattern modes.
[0:18:02.328947861] [2151]  INFO RPI raspberrypi.cpp:1352 Registered camera /base/soc/i2c0mux/i2c@1/imx477@1a to Unicam device /dev/media1 and ISP device /dev/media0


## Method #1: Using the set_controls function.

In [2]:
my_controls = {'NoiseReductionMode': 0,
               'Contrast': 0,
               'Sharpness': 0}

picam2.set_controls(my_controls)

The `Controls` class is automatically brought into the `Picamera2` class, so for most applications you probably won't need to
call it individually. It is renamed to `controls` when using it through the `Picamera2`.
To check the controls, you can use `controls.config`. The nice thing about this is that the function automatically compiles the controls you adjusted into a dictionary and checks their validity (within reason). Please keep in mind that these controls are not applied until the camera starts or is already started. If you query this function while it is already started, it will always return empty. To get the current settings at that point, you would use `controls.current`.

In [3]:
picam2.controls.config

{'Contrast': 0, 'NoiseReductionMode': 0, 'Sharpness': 0}

## Method #2: Using the controls option in a prebuilt configuration.

In [4]:
my_controls = {'NoiseReductionMode': 1,
               'Contrast': 1,
               'Sharpness': 1}

cfg = picam2.preview_configuration(controls = my_controls)
picam2.configure(cfg)
picam2.controls.config

[0:18:02.452312723] [2137]  INFO Camera camera.cpp:1029 configuring streams: (0) 640x480-XBGR8888
[0:18:02.452867051] [2151]  INFO RPI raspberrypi.cpp:760 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 2028x1520-SBGGR12_1X12 - Selected unicam format: 2028x1520-pBCC


{'Contrast': 1, 'NoiseReductionMode': 1, 'Sharpness': 1}

## Method #3: Using class attributes.
Controls are imported from the `Controls` class, which treats each control as an attribute or property of the class.

In [5]:
picam2.controls.NoiseReductionMode = 2
picam2.controls.Contrast = 1
picam2.controls.Sharpness = 1
picam2.controls.config

{'Contrast': 1, 'NoiseReductionMode': 2, 'Sharpness': 1}

## Control Input Options

In most cases, `the Controls` class will notify you if the control you input is the incorrect value or type when
the camera reads in the controls. There are several ways you can input values for each control, but it depends on the control.

As of writing this notebook, control names and enums must match the syntax of what libcamera uses.
For example, if you input "off", an error will be thrown by libcamera. The correct input would be "Off".
Another example would `picam2.controls.noisereductionmode` which would not register as a control option. The correct input would be `picam2.controls.NoiseReductionMode`.

For controls that configure camera modes, you can supply an integer, an enum, or string.

In [6]:
#These all turn off NoiseReductionMode.
picam2.controls.NoiseReductionMode = 0
picam2.controls.NoiseReductionMode = libcamera.NoiseReductionMode.Off
picam2.controls.NoiseReductionMode = 'Off'

For controls that enable or disable a function, you can supply an integer or bool.

In [7]:
#These lines disable auto exposure mode.

picam2.controls.AeEnable = 0
picam2.controls.AeEnable = False

For controls that require an array as input, you can supply it as a tuple or a list.

In [8]:
#These both set the FrameDurationLimits to 33333 or 30 frames per second.

picam2.controls.FrameDurationLimits = (33333,33333)
picam2.controls.FrameDurationLimits = [33333,33333]

## Configuring the Controls
At this stage, we actually haven't sent the controls to the camera. The neat thing is that we don't
need to do this. This is done automatically when you tell the camera to start.

In [9]:
picam2.controls._blank #You may not need this. It only clears the controls BEFORE the camera is started.
picam2.configure(picam2.preview_configuration())
picam2.controls.NoiseReductionMode = 2  #Need to put it after camera configuration because of how preview_configuration is built.
picam2.start_preview(Preview.QTGL)
picam2.start()

[0:18:02.619273177] [2137]  INFO Camera camera.cpp:1029 configuring streams: (0) 640x480-XBGR8888
[0:18:02.619752544] [2151]  INFO RPI raspberrypi.cpp:760 Sensor: /base/soc/i2c0mux/i2c@1/imx477@1a - Selected sensor format: 2028x1520-SBGGR12_1X12 - Selected unicam format: 2028x1520-pBCC
[0:18:04.391050799] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: NoiseReductionMode = 2
qt.qpa.xcb: QXcbConnection: XCB error: 148 (Unknown), sequence: 192, resource id: 0, major code: 140 (Unknown), minor code: 20


## Control Manipulation

All of the controls are technically class variables. That means that all instances of the class share the same values.
So controls can be updated by either method after the camera has already started, without having to start the camera again.

In [10]:
for i in range(0,20):
    j = i/20 + 0.05
    picam2.controls.Brightness = j
    time.sleep(1)

[0:18:05.275939471] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: NoiseReductionMode = 2
[0:18:05.276047118] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.050000
[0:18:06.004939293] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.100000
[0:18:07.006321133] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.150000
[0:18:08.002098042] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.200000
[0:18:09.005321521] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.250000
[0:18:10.003580102] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.300000
[0:18:11.005595673] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.350000
[0:18:12.004193462] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.400000
[0:18:13.003264085] [2159]  INFO IPARPI raspberrypi.cpp:635 Request ctrl: Brightness = 0.450000
[0:18:14.003655847] [2159]  INFO IPARPI

## Current Settings

After the settings that you assign are passed to the camera, the `Controls`
class is effectively reset so that there is no risk of repetition. However,
it does keep track of the current settings of the camera.
The current settings are always the settings that you last sent to the camera and that were accepted.

If we look at the current settings now, we should see that the NoiseReductionMode is set to 2
and the Brightness is 1.

In [11]:
picam2.stop_preview()
picam2.controls.current

{'Brightness': 1.0, 'NoiseReductionMode': 2}

## Resetting Controls (in development)

If you wanted to reset the controls to what they were when you first turned on the camera (i.e. the defaults),
you can issue the `reset` command, which retains the default controls when `Picamera2` was instantiated. If you compare the libcamera output of the `reset` function to the `ranges` function, you may notice that some values are 0 when the 0 is technically outside the limits. This just means that the camera is handling these internally or in an automatic mode.

In [12]:
picam2.controls.defaults #This is a read-only property and will reflect whatever comes out of Picamera2().camera.controls.

{'AeConstraintMode': 0,
 'AeEnable': 0,
 'AeExposureMode': 0,
 'AeMeteringMode': 0,
 'AnalogueGain': 0,
 'AwbEnable': 0,
 'AwbMode': 0,
 'Brightness': 0.0,
 'ColourCorrectionMatrix': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ColourGains': [0, 0],
 'Contrast': 1.0,
 'ExposureTime': 0,
 'ExposureValue': 0.0,
 'FrameDurationLimits': [0, 0],
 'NoiseReductionMode': 0,
 'Saturation': 1.0,
 'ScalerCrop': (0, 0, 0, 0),
 'Sharpness': 1.0}

In [13]:
picam2.controls.reset #Resets the camera to the controls it had when it was first turned on. Read only. Accepts no inputs.
picam2.controls.config

{'AeConstraintMode': 0,
 'AeEnable': 0,
 'AeExposureMode': 0,
 'AeMeteringMode': 0,
 'AnalogueGain': 0,
 'AwbEnable': 0,
 'AwbMode': 0,
 'Brightness': 0.0,
 'ColourCorrectionMatrix': [0, 0, 0, 0, 0, 0, 0, 0, 0],
 'ColourGains': [0, 0],
 'Contrast': 1.0,
 'ExposureTime': 0,
 'ExposureValue': 0.0,
 'FrameDurationLimits': [0, 0],
 'NoiseReductionMode': 0,
 'Saturation': 1.0,
 'ScalerCrop': (0, 0, 0, 0),
 'Sharpness': 1.0}

We can see that the changes in the output reflect the values in the defaults.

## Control Ranges (in development)
To get an idea of the valid ranges for each control, you can query the ranges.

In [14]:
picam2.controls.ranges #Read only.

{'AeConstraintMode': {'min': 0, 'max': 3},
 'AeEnable': {'min': False, 'max': True},
 'AeExposureMode': {'min': 0, 'max': 3},
 'AeMeteringMode': {'min': 0, 'max': 3},
 'AnalogueGain': {'min': 1.0, 'max': 32.0},
 'AwbEnable': {'min': False, 'max': True},
 'AwbMode': {'min': 0, 'max': 7},
 'Brightness': {'min': -1.0, 'max': 1.0},
 'ColourCorrectionMatrix': {'min': -16.0, 'max': 16.0},
 'ColourGains': {'min': 0.0, 'max': 32.0},
 'Contrast': {'min': 0.0, 'max': 32.0},
 'ExposureTime': {'min': 0, 'max': 999999},
 'ExposureValue': {'min': -8.0, 'max': 8.0},
 'FrameDurationLimits': {'min': 1000, 'max': 1000000000},
 'NoiseReductionMode': {'min': 0, 'max': 4},
 'Saturation': {'min': 0.0, 'max': 32.0},
 'ScalerCrop': {'min': (0, 0, 0, 0), 'max': (65535, 65535, 65535, 65535)},
 'Sharpness': {'min': 0.0, 'max': 16.0}}

## End of Example

In [15]:
picam2.stop()
picam2.close()